In [2]:
# DB 방식으로 변경: CSV 읽어서 documents 테이블의 사원 정보 업데이트
# activities, referrers JSON 업데이트 + drafter 단일 컬럼 업데이트
"""
 [기능]
 CSV 파일의 인사정보를 읽어서 DB documents 테이블의 사원 정보 업데이트
 (특정 연도 문서만 대상)

 [입력]
 - CSV 파일: 인사정보_부서코드추가.csv
   (사원명, ID, 부서, 사원번호, 직위, 부서코드)

 [업데이트 대상]
 - drafter 단일 컬럼들: drafter_position, drafter_dept, 
   drafter_email, drafter_dept_code
 - activities JSON: 결재자별 emailId, deptName, positionName, deptCode
 - referrers JSON: empNo, deptCode

 [현직자/퇴사자 처리]
 - 현직자: CSV 정보로 정상 업데이트
 - 퇴사자 (CSV에 없는 이름):
   - drafter: email을 'master'로 설정, 나머지 공란
   - activities: 이름만 남기고 모두 공란
   - referrers: 이름만 남기고 모두 공란

 [동작 순서]
 1. CSV 파일 읽어서 사원명 → 정보 딕셔너리 생성
 2. DB에서 특정 연도(end_year) 문서 조회
 3. 각 문서별로:
    - drafter 정보 업데이트
    - activities JSON 파싱 → 업데이트 → 직렬화
    - referrers JSON 파싱 → 업데이트 → 직렬화
 4. DB UPDATE 실행 및 커밋

 [설정 (#여기를 수정하세요)]
 - DB_CONFIG: DB 접속 정보
 - CSV_FILE: 인사정보 CSV 파일 경로
 - WHERE end_year = 2010: 대상 연도 (SQL 직접 수정. ctrl+f 해서 2010을 원하는 연도로 모두 바꾸기 하기)

 [의존성]
 - pandas
 - pymysql
 -> pip install pandas pymysql
"""
import pandas as pd
import json
import pymysql

# ═══════════════════════════════════════════════════════════
# 설정 #여기를 수정하세요
# ═══════════════════════════════════════════════════════════

# DB 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval'
}

CSV_FILE = '인사정보_부서코드추가.csv'

# ═══════════════════════════════════════════════════════════
# 1. CSV 파일 읽기
# ═══════════════════════════════════════════════════════════

print("=" * 70)
print("📊 CSV 파일 읽기 시작")
print("=" * 70)

df = pd.read_csv(CSV_FILE, encoding='utf-8-sig')

# 사원명을 키로 하는 딕셔너리 생성
employee_dict = {}
for _, row in df.iterrows():
    employee_dict[row['사원명']] = {
        'emailId': row['ID'],
        'deptName': row['부서'],
        'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
        'positionName': row['직위'] if pd.notna(row['직위']) else '',
        'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''
    }

print(f"✅ 총 {len(employee_dict)}명의 사원 정보 로드 완료\n")

# ═══════════════════════════════════════════════════════════
# 2. 업데이트 함수들 (기존과 동일)
# ═══════════════════════════════════════════════════════════

# activities용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_activity_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: 정상 업데이트
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['emailId'] = ''
            person_obj['deptName'] = ''
            person_obj['positionName'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# referrers용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_referrer_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: empNo와 deptCode 업데이트
            person_obj['empNo'] = employee_dict[name]['empNo']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['empNo'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# ═══════════════════════════════════════════════════════════
# 3. DB 연결 및 데이터 읽기
# ═══════════════════════════════════════════════════════════

print("=" * 70)
print("💾 DB 연결 및 데이터 읽기")
print("=" * 70)

try:
    # DB 연결
    conn = pymysql.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database'],
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    print(f"✅ DB 연결 성공: {DB_CONFIG['host']}:{DB_CONFIG['database']}\n")
    
    cursor = conn.cursor()
    
    # documents 테이블 전체 읽기
    select_sql = """
    SELECT id, source_id, drafter_name, drafter_position, drafter_dept, 
           drafter_email, drafter_dept_code,
           activities, referrers, end_year
    FROM documents
    """
    
    cursor.execute(select_sql)
    rows = cursor.fetchall()
    
    print(f"✅ 총 {len(rows)}건의 문서 로드 완료\n")
    
    # ═══════════════════════════════════════════════════════════
    # 4. 데이터 업데이트 처리
    # ═══════════════════════════════════════════════════════════
    
    print("=" * 70)
    print("🔄 사원 정보 업데이트 시작")
    print("=" * 70)
    
    update_sql = """
    UPDATE documents 
    SET drafter_position = %s,
        drafter_dept = %s,
        drafter_email = %s,
        drafter_dept_code = %s,
        activities = %s,
        referrers = %s
    WHERE id = %s
    """
    
    updated_count = 0
    error_count = 0
    
    for idx, row in enumerate(rows, 1):
        try:
            doc_id = row['id']
            drafter_name = row['drafter_name']
            
            # drafter 정보 업데이트 (퇴사자 특별 처리!)
            if drafter_name and drafter_name in employee_dict:
                # 현직자: 정상 업데이트
                drafter_position = employee_dict[drafter_name]['positionName']
                drafter_dept = employee_dict[drafter_name]['deptName']
                drafter_email = employee_dict[drafter_name]['emailId']
                drafter_dept_code = employee_dict[drafter_name]['deptCode']
            else:
                # 퇴사자 또는 이름 없음: 특별 처리!
                drafter_position = ''
                drafter_dept = ''
                drafter_email = 'master'  # ← 퇴사자는 master!
                drafter_dept_code = ''
            
            # activities JSON 업데이트
            activities_text = row.get('activities', '[]')
            try:
                activities = json.loads(activities_text) if activities_text else []
            except:
                activities = []
            
            for activity in activities:
                update_activity_info(activity)
            
            updated_activities = json.dumps(activities, ensure_ascii=False)
            
            # referrers JSON 업데이트
            referrers_text = row.get('referrers', '[]')
            try:
                referrers = json.loads(referrers_text) if referrers_text else []
            except:
                referrers = []
            
            for referrer in referrers:
                update_referrer_info(referrer)
            
            updated_referrers = json.dumps(referrers, ensure_ascii=False)
            
            # UPDATE 실행
            cursor.execute(update_sql, (
                drafter_position,
                drafter_dept,
                drafter_email,
                drafter_dept_code,
                updated_activities,
                updated_referrers,
                doc_id
            ))
            
            updated_count += 1
            
            # 진행 상황 출력 (100건마다)
            if idx % 100 == 0:
                print(f"  진행 중... {idx}/{len(rows)} (성공: {updated_count}, 실패: {error_count})")
            
        except Exception as e:
            error_count += 1
            if error_count <= 5:
                print(f"  ❌ 오류 (ID: {row.get('id')}): {str(e)[:100]}")
    
    # 커밋
    print("\n💾 변경사항 커밋 중...")
    conn.commit()
    print("✅ 커밋 완료\n")
    
    # ═══════════════════════════════════════════════════════════
    # 5. 결과 요약
    # ═══════════════════════════════════════════════════════════
    
    print("=" * 70)
    print("✅ 업데이트 완료!")
    print("=" * 70)
    print(f"총 문서: {len(rows)}건")
    print(f"성공: {updated_count}건")
    print(f"실패: {error_count}건")
    print("=" * 70)
    
    # 샘플 확인
    if rows:
        cursor.execute("""
        SELECT id, source_id, drafter_name, drafter_position, drafter_dept, 
               drafter_email, drafter_dept_code,
               activities, referrers
        FROM documents
        LIMIT 1
        """)
        sample = cursor.fetchone()
        
        print("\n📋 업데이트 결과 샘플 (첫 번째 문서):")
        print(f"  ID: {sample['id']}")
        print(f"  Source ID: {sample['source_id']}")
        print(f"  기안자: {sample['drafter_name']} / {sample['drafter_position']} / {sample['drafter_dept']}")
        print(f"  기안자 이메일: {sample['drafter_email']}")
        print(f"  기안자 부서코드: {sample['drafter_dept_code']}")
        
        # activities 샘플
        try:
            activities = json.loads(sample['activities']) if sample['activities'] else []
            if activities:
                print(f"\n  Activities 첫 번째:")
                print(f"    {json.dumps(activities[0], ensure_ascii=False, indent=4)}")
        except:
            pass
        
        # referrers 샘플
        try:
            referrers = json.loads(sample['referrers']) if sample['referrers'] else []
            if referrers:
                print(f"\n  Referrers 첫 번째:")
                print(f"    {json.dumps(referrers[0], ensure_ascii=False, indent=4)}")
        except:
            pass
    
except pymysql.Error as e:
    print(f"\n❌ DB 오류 발생:")
    print(f"  Error Code: {e.args[0]}")
    print(f"  Error Message: {e.args[1]}")

except Exception as e:
    print(f"\n❌ 일반 오류 발생: {e}")
    import traceback
    traceback.print_exc()

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
        print("\n✅ DB 연결 종료")

print("\n" + "=" * 70)
print("🎉 프로그램 종료")
print("=" * 70)

📊 CSV 파일 읽기 시작
✅ 총 156명의 사원 정보 로드 완료

💾 DB 연결 및 데이터 읽기
✅ DB 연결 성공: localhost:any_approval

✅ 총 8175건의 문서 로드 완료

🔄 사원 정보 업데이트 시작
  진행 중... 100/8175 (성공: 100, 실패: 0)
  진행 중... 200/8175 (성공: 200, 실패: 0)
  진행 중... 300/8175 (성공: 300, 실패: 0)
  진행 중... 400/8175 (성공: 400, 실패: 0)
  진행 중... 500/8175 (성공: 500, 실패: 0)
  진행 중... 600/8175 (성공: 600, 실패: 0)
  진행 중... 700/8175 (성공: 700, 실패: 0)
  진행 중... 800/8175 (성공: 800, 실패: 0)
  진행 중... 900/8175 (성공: 900, 실패: 0)
  진행 중... 1000/8175 (성공: 1000, 실패: 0)
  진행 중... 1100/8175 (성공: 1100, 실패: 0)
  진행 중... 1200/8175 (성공: 1200, 실패: 0)
  진행 중... 1300/8175 (성공: 1300, 실패: 0)
  진행 중... 1400/8175 (성공: 1400, 실패: 0)
  진행 중... 1500/8175 (성공: 1500, 실패: 0)
  진행 중... 1600/8175 (성공: 1600, 실패: 0)
  진행 중... 1700/8175 (성공: 1700, 실패: 0)
  진행 중... 1800/8175 (성공: 1800, 실패: 0)
  진행 중... 1900/8175 (성공: 1900, 실패: 0)
  진행 중... 2000/8175 (성공: 2000, 실패: 0)
  진행 중... 2100/8175 (성공: 2100, 실패: 0)
  진행 중... 2200/8175 (성공: 2200, 실패: 0)
  진행 중... 2300/8175 (성공: 2300, 실패: 0)
  진행 중... 2400/8

In [1]:
# DB 방식으로 변경: CSV 읽어서 documents 테이블의 사원 정보 업데이트
# activities, referrers JSON 업데이트 + drafter 단일 컬럼 업데이트

import pandas as pd
import json
import pymysql

# ═══════════════════════════════════════════════════════════
# 설정
# ═══════════════════════════════════════════════════════════

# DB 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval'
}

CSV_FILE = '인사정보_부서코드추가.csv'

# ═══════════════════════════════════════════════════════════
# 1. CSV 파일 읽기
# ═══════════════════════════════════════════════════════════

print("=" * 70)
print("📊 CSV 파일 읽기 시작")
print("=" * 70)

df = pd.read_csv(CSV_FILE, encoding='utf-8-sig')

# 사원명을 키로 하는 딕셔너리 생성
employee_dict = {}
for _, row in df.iterrows():
    employee_dict[row['사원명']] = {
        'emailId': row['ID'],
        'deptName': row['부서'],
        'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
        'positionName': row['직위'] if pd.notna(row['직위']) else '',
        'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''
    }

print(f"✅ 총 {len(employee_dict)}명의 사원 정보 로드 완료\n")

# ═══════════════════════════════════════════════════════════
# 2. 업데이트 함수들 (기존과 동일)
# ═══════════════════════════════════════════════════════════

# activities용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_activity_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: 정상 업데이트
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['emailId'] = ''
            person_obj['deptName'] = ''
            person_obj['positionName'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# referrers용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_referrer_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: empNo와 deptCode 업데이트
            person_obj['empNo'] = employee_dict[name]['empNo']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['empNo'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# ═══════════════════════════════════════════════════════════
# 3. DB 연결 및 데이터 읽기
# ═══════════════════════════════════════════════════════════

print("=" * 70)
print("💾 DB 연결 및 데이터 읽기")
print("=" * 70)

try:
    # DB 연결
    conn = pymysql.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database'],
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    print(f"✅ DB 연결 성공: {DB_CONFIG['host']}:{DB_CONFIG['database']}\n")
    
    cursor = conn.cursor()
    
    # documents 테이블 전체 읽기
    select_sql = """
    SELECT id, source_id, drafter_name, drafter_position, drafter_dept, 
           activities, referrers, end_year
    FROM documents
    """
    
    cursor.execute(select_sql)
    rows = cursor.fetchall()
    
    print(f"✅ 총 {len(rows)}건의 문서 로드 완료\n")
    
    # ═══════════════════════════════════════════════════════════
    # 4. 데이터 업데이트 처리
    # ═══════════════════════════════════════════════════════════
    
    print("=" * 70)
    print("🔄 사원 정보 업데이트 시작")
    print("=" * 70)
    
    update_sql = """
    UPDATE documents 
    SET drafter_position = %s,
        drafter_dept = %s,
        activities = %s,
        referrers = %s
    WHERE id = %s
    """
    
    updated_count = 0
    error_count = 0
    
    for idx, row in enumerate(rows, 1):
        try:
            doc_id = row['id']
            drafter_name = row['drafter_name']
            
            # drafter 정보 업데이트 (단일 컬럼)
            if drafter_name and drafter_name in employee_dict:
                # 현직자
                drafter_position = employee_dict[drafter_name]['positionName']
                drafter_dept = employee_dict[drafter_name]['deptName']
            else:
                # 퇴사자 또는 이름 없음
                drafter_position = ''
                drafter_dept = ''
            
            # activities JSON 업데이트
            activities_text = row.get('activities', '[]')
            try:
                activities = json.loads(activities_text) if activities_text else []
            except:
                activities = []
            
            for activity in activities:
                update_activity_info(activity)
            
            updated_activities = json.dumps(activities, ensure_ascii=False)
            
            # referrers JSON 업데이트
            referrers_text = row.get('referrers', '[]')
            try:
                referrers = json.loads(referrers_text) if referrers_text else []
            except:
                referrers = []
            
            for referrer in referrers:
                update_referrer_info(referrer)
            
            updated_referrers = json.dumps(referrers, ensure_ascii=False)
            
            # UPDATE 실행
            cursor.execute(update_sql, (
                drafter_position,
                drafter_dept,
                updated_activities,
                updated_referrers,
                doc_id
            ))
            
            updated_count += 1
            
            # 진행 상황 출력 (100건마다)
            if idx % 100 == 0:
                print(f"  진행 중... {idx}/{len(rows)} (성공: {updated_count}, 실패: {error_count})")
            
        except Exception as e:
            error_count += 1
            if error_count <= 5:
                print(f"  ❌ 오류 (ID: {row.get('id')}): {str(e)[:100]}")
    
    # 커밋
    print("\n💾 변경사항 커밋 중...")
    conn.commit()
    print("✅ 커밋 완료\n")
    
    # ═══════════════════════════════════════════════════════════
    # 5. 결과 요약
    # ═══════════════════════════════════════════════════════════
    
    print("=" * 70)
    print("✅ 업데이트 완료!")
    print("=" * 70)
    print(f"총 문서: {len(rows)}건")
    print(f"성공: {updated_count}건")
    print(f"실패: {error_count}건")
    print("=" * 70)
    
    # 샘플 확인
    if rows:
        cursor.execute("""
        SELECT id, source_id, drafter_name, drafter_position, drafter_dept, 
               activities, referrers
        FROM documents
        LIMIT 1
        """)
        sample = cursor.fetchone()
        
        print("\n📋 업데이트 결과 샘플 (첫 번째 문서):")
        print(f"  ID: {sample['id']}")
        print(f"  Source ID: {sample['source_id']}")
        print(f"  기안자: {sample['drafter_name']} / {sample['drafter_position']} / {sample['drafter_dept']}")
        
        # activities 샘플
        try:
            activities = json.loads(sample['activities']) if sample['activities'] else []
            if activities:
                print(f"\n  Activities 첫 번째:")
                print(f"    {json.dumps(activities[0], ensure_ascii=False, indent=4)}")
        except:
            pass
        
        # referrers 샘플
        try:
            referrers = json.loads(sample['referrers']) if sample['referrers'] else []
            if referrers:
                print(f"\n  Referrers 첫 번째:")
                print(f"    {json.dumps(referrers[0], ensure_ascii=False, indent=4)}")
        except:
            pass
    
except pymysql.Error as e:
    print(f"\n❌ DB 오류 발생:")
    print(f"  Error Code: {e.args[0]}")
    print(f"  Error Message: {e.args[1]}")

except Exception as e:
    print(f"\n❌ 일반 오류 발생: {e}")
    import traceback
    traceback.print_exc()

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
        print("\n✅ DB 연결 종료")

print("\n" + "=" * 70)
print("🎉 프로그램 종료")
print("=" * 70)

📊 CSV 파일 읽기 시작
✅ 총 156명의 사원 정보 로드 완료

💾 DB 연결 및 데이터 읽기
✅ DB 연결 성공: localhost:any_approval

✅ 총 8175건의 문서 로드 완료

🔄 사원 정보 업데이트 시작
  진행 중... 100/8175 (성공: 100, 실패: 0)
  진행 중... 200/8175 (성공: 200, 실패: 0)
  진행 중... 300/8175 (성공: 300, 실패: 0)
  진행 중... 400/8175 (성공: 400, 실패: 0)
  진행 중... 500/8175 (성공: 500, 실패: 0)
  진행 중... 600/8175 (성공: 600, 실패: 0)
  진행 중... 700/8175 (성공: 700, 실패: 0)
  진행 중... 800/8175 (성공: 800, 실패: 0)
  진행 중... 900/8175 (성공: 900, 실패: 0)
  진행 중... 1000/8175 (성공: 1000, 실패: 0)
  진행 중... 1100/8175 (성공: 1100, 실패: 0)
  진행 중... 1200/8175 (성공: 1200, 실패: 0)
  진행 중... 1300/8175 (성공: 1300, 실패: 0)
  진행 중... 1400/8175 (성공: 1400, 실패: 0)
  진행 중... 1500/8175 (성공: 1500, 실패: 0)
  진행 중... 1600/8175 (성공: 1600, 실패: 0)
  진행 중... 1700/8175 (성공: 1700, 실패: 0)
  진행 중... 1800/8175 (성공: 1800, 실패: 0)
  진행 중... 1900/8175 (성공: 1900, 실패: 0)
  진행 중... 2000/8175 (성공: 2000, 실패: 0)
  진행 중... 2100/8175 (성공: 2100, 실패: 0)
  진행 중... 2200/8175 (성공: 2200, 실패: 0)
  진행 중... 2300/8175 (성공: 2300, 실패: 0)
  진행 중... 2400/8

In [1]:
# 이건 activities의 기안자 master 처리 안함 - 이게 맞음!

#기틀 json의 유저 정보를 변경된 유저정보로 교체
# approvals 이슈 수정
# 퇴사자 처리 로직 추가
# 부서번호 추가 코드

import pandas as pd
import json

# CSV 파일 읽기
df = pd.read_csv('인사정보_부서코드추가.csv', encoding='utf-8-sig')

# 사원명을 키로 하는 딕셔너리 생성 (부서코드 추가!)
employee_dict = {}
for _, row in df.iterrows():
    employee_dict[row['사원명']] = {
        'emailId': row['ID'],
        'deptName': row['부서'],
        'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
        'positionName': row['직위'] if pd.notna(row['직위']) else '',
        'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''  # 부서코드 추가!
    }

print(f"총 {len(employee_dict)}명의 사원 정보 로드 완료")

# JSON 파일 읽기
#컨트롤러
with open('three_ispublic_insert.json', 'r', encoding='utf-8') as f:
    approval_list = json.load(f)

print(f"총 {len(approval_list)}개의 결재 문서 로드 완료")

# 기안자용 업데이트 함수 (퇴사자는 emailId를 master로)
def update_drafter_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: 정상 업데이트
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: emailId만 master, 나머지 공란
            person_obj['emailId'] = 'master'
            person_obj['deptName'] = ''
            person_obj['positionName'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# activities용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_activity_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: 정상 업데이트
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['emailId'] = ''
            person_obj['deptName'] = ''
            person_obj['positionName'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# referrers용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_referrer_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: empNo와 deptCode 업데이트
            person_obj['empNo'] = employee_dict[name]['empNo']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['empNo'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# 결재 목록 순회하며 정보 업데이트
updated_count = 0
retired_count = 0  # 퇴사자 카운트

for doc in approval_list:
    # drafter 정보 업데이트 (퇴사자는 emailId를 master로)
    if 'drafter' in doc:
        doc['drafter'] = update_drafter_info(doc['drafter'])
        updated_count += 1
    
    # referrers 리스트의 각 참조자 정보 업데이트
    if 'referrers' in doc:
        for referrer in doc['referrers']:
            update_referrer_info(referrer)
            updated_count += 1
    
    # activities 리스트의 각 활동자 정보 업데이트 (퇴사자는 이름만 남기고 공란)
    if 'activities' in doc:
        for activity in doc['activities']:
            update_activity_info(activity)
            updated_count += 1

print(f"총 {updated_count}개의 사원 정보 업데이트 완료")

# 결과를 새로운 JSON 파일로 저장
#컨트롤러
with open('three_newuser_insert.json', 'w', encoding='utf-8') as f:
    json.dump(approval_list, f, ensure_ascii=False, indent=2)

print("three_newuser_insert.json 파일로 저장 완료")

# 업데이트 결과 샘플 확인
print("\n첫 번째 문서의 drafter 정보:")
print(json.dumps(approval_list[0]['drafter'], ensure_ascii=False, indent=2))

print("\n첫 번째 문서의 첫 번째 activity 정보:")
if approval_list[0].get('activities'):
    print(json.dumps(approval_list[0]['activities'][0], ensure_ascii=False, indent=2))

if approval_list[0].get('referrers'):
    print("\n첫 번째 문서의 첫 번째 referrer 정보:")
    print(json.dumps(approval_list[0]['referrers'][0], ensure_ascii=False, indent=2))

총 156명의 사원 정보 로드 완료
총 8175개의 결재 문서 로드 완료
총 41512개의 사원 정보 업데이트 완료
three_newuser_insert.json 파일로 저장 완료

첫 번째 문서의 drafter 정보:
{
  "positionName": "상무",
  "deptName": "ITO사업팀",
  "name": "고상환",
  "emailId": "shko",
  "deptCode": "DB60"
}

첫 번째 문서의 첫 번째 activity 정보:
{
  "positionName": "상무",
  "deptName": "ITO사업팀",
  "actionLogType": "DRAFT",
  "name": "고상환",
  "emailId": "shko",
  "type": "DRAFT",
  "actionDate": 1609740886000,
  "deptCode": "DB60",
  "actionComment": "LGC 실험실행관리시스템 계약 품의입니다."
}

첫 번째 문서의 첫 번째 referrer 정보:
{
  "name": "김민서",
  "empNo": "201602881005",
  "deptCode": "AB30"
}


In [ ]:
# ????기안자라는게 activities에서의 기안자는 공란인건지 거기서도 master로 넣어야하는지???????????????/
# 이건 activities의 기안자도 master 처리 함

#기틀 json의 유저 정보를 변경된 유저정보로 교체
# approvals 이슈 수정
# 퇴사자 처리 로직 추가 (activities의 DRAFT도 master 처리)
# 부서번호 추가 코드

import pandas as pd
import json

# CSV 파일 읽기
df = pd.read_csv('인사정보_부서코드추가.csv', encoding='utf-8-sig')

# 사원명을 키로 하는 딕셔너리 생성 (부서코드 추가!)
employee_dict = {}
for _, row in df.iterrows():
    employee_dict[row['사원명']] = {
        'emailId': row['ID'],
        'deptName': row['부서'],
        'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
        'positionName': row['직위'] if pd.notna(row['직위']) else '',
        'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''  # 부서코드 추가!
    }

print(f"총 {len(employee_dict)}명의 사원 정보 로드 완료")

# JSON 파일 읽기
with open('use_mapping_test.json', 'r', encoding='utf-8') as f:
    approval_list = json.load(f)

print(f"총 {len(approval_list)}개의 결재 문서 로드 완료")

# 기안자용 업데이트 함수 (퇴사자는 emailId를 master로)
def update_drafter_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: 정상 업데이트
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: emailId만 master, 나머지 공란
            person_obj['emailId'] = 'master'
            person_obj['deptName'] = ''
            person_obj['positionName'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# activities용 업데이트 함수 (DRAFT 퇴사자는 master, 나머지는 공란)
def update_activity_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: 정상 업데이트
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: type에 따라 다르게 처리
            if person_obj.get('type') == 'DRAFT':
                # DRAFT(기안자)는 emailId를 master로
                person_obj['emailId'] = 'master'
            else:
                # APPROVAL, AGREEMENT 등은 공란
                person_obj['emailId'] = ''
            person_obj['deptName'] = ''
            person_obj['positionName'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# referrers용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_referrer_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: empNo와 deptCode 업데이트
            person_obj['empNo'] = employee_dict[name]['empNo']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['empNo'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# 결재 목록 순회하며 정보 업데이트
updated_count = 0

for doc in approval_list:
    # drafter 정보 업데이트 (퇴사자는 emailId를 master로)
    if 'drafter' in doc:
        doc['drafter'] = update_drafter_info(doc['drafter'])
        updated_count += 1
    
    # referrers 리스트의 각 참조자 정보 업데이트
    if 'referrers' in doc:
        for referrer in doc['referrers']:
            update_referrer_info(referrer)
            updated_count += 1
    
    # activities 리스트의 각 활동자 정보 업데이트 (DRAFT 퇴사자는 master, 나머지는 공란)
    if 'activities' in doc:
        for activity in doc['activities']:
            update_activity_info(activity)
            updated_count += 1

print(f"총 {updated_count}개의 사원 정보 업데이트 완료")

# 결과를 새로운 JSON 파일로 저장
with open('결재목록_업데이트완료.json', 'w', encoding='utf-8') as f:
    json.dump(approval_list, f, ensure_ascii=False, indent=2)

print("결재목록_업데이트완료.json 파일로 저장 완료")

# 업데이트 결과 샘플 확인
print("\n첫 번째 문서의 drafter 정보:")
print(json.dumps(approval_list[0]['drafter'], ensure_ascii=False, indent=2))

print("\n첫 번째 문서의 첫 번째 activity 정보:")
if approval_list[0].get('activities'):
    print(json.dumps(approval_list[0]['activities'][0], ensure_ascii=False, indent=2))

if approval_list[0].get('referrers'):
    print("\n첫 번째 문서의 첫 번째 referrer 정보:")
    print(json.dumps(approval_list[0]['referrers'][0], ensure_ascii=False, indent=2))